In [1]:
import coiled
from dask.distributed import Client
import dask

In [2]:
import dask.dataframe as dd

In [3]:
cluster = coiled.Cluster(n_workers = 20)
client = cluster.get_client()

Output()

Output()

In [4]:
client

<Client: 'tls://10.0.33.27:8786' processes=19 threads=76, memory=282.03 GiB>

In [5]:
dask.config.set({"dataframe.convert-string": True})

In [6]:
df = dd.read_parquet(
    "s3://coiled-datasets/uber-lyft-tlc/",
    storage_options={'anon': True}
)

In [7]:
df.dtypes

hvfhs_license_num       string[pyarrow]
dispatching_base_num    string[pyarrow]
originating_base_num    string[pyarrow]
request_datetime         datetime64[ns]
on_scene_datetime        datetime64[ns]
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
PULocationID                      int32
DOLocationID                      int32
trip_miles                      float32
trip_time                         int32
base_passenger_fare             float32
tolls                           float32
bcf                             float32
sales_tax                       float32
congestion_surcharge            float32
airport_fee                     float32
tips                            float32
driver_pay                      float32
shared_request_flag            category
shared_match_flag              category
access_a_ride_flag             category
wav_request_flag               category
wav_match_flag                 category
dtype: object

In [8]:
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02867,B02867,2019-02-01 00:01:26,2019-02-01 00:02:55,2019-02-01 00:05:18,2019-02-01 00:14:57,245,251,2.45,...,0.83,0.0,NaN,0.0,7.480000,Y,N,N,N,NaN
1,HV0003,B02879,B02879,2019-02-01 00:26:08,2019-02-01 00:41:29,2019-02-01 00:41:29,2019-02-01 00:49:39,216,197,1.71,...,0.70,0.0,NaN,2.0,7.930000,N,N,N,N,NaN
2,HV0005,B02510,<NA>,2019-02-01 00:48:58,NaT,2019-02-01 00:51:34,2019-02-01 01:28:29,261,234,5.01,...,3.99,0.0,NaN,0.0,35.970001,N,Y,N,N,NaN
3,HV0005,B02510,<NA>,2019-02-01 00:02:15,NaT,2019-02-01 00:03:51,2019-02-01 00:07:16,87,87,0.34,...,0.64,0.0,NaN,3.0,5.390000,N,Y,N,N,NaN
4,HV0005,B02510,<NA>,2019-02-01 00:06:17,NaT,2019-02-01 00:09:44,2019-02-01 00:39:56,87,198,6.84,...,2.16,0.0,NaN,4.0,17.070000,N,Y,N,N,NaN


In [16]:
dask.utils.format_bytes(
    df.memory_usage(deep=True).sum().compute()
)

'88.65 GiB'

In [20]:
df_sample = df.sample(frac=0.1)

In [21]:
df_sample = df_sample.persist()

In [19]:
dask.utils.format_bytes(
    df_sample.memory_usage(deep=True).sum().compute()
)

'4.44 GiB'

In [22]:
from dask.sizeof import sizeof

In [23]:
partitions_mem_stats = df_sample.map_partitions(sizeof).compute()

In [24]:
(partitions_mem_stats / 1024**2).describe() #in MiB

count    720.000000
mean      12.642363
std        0.246717
min       11.779142
25%       12.438430
50%       12.625019
75%       12.895652
max       13.024080
dtype: float64

In [25]:
dask.utils.format_bytes(df_sample.partitions[0].memory_usage(deep=True).compute().sum())

'13.00 MiB'

In [26]:
%%time
#passanger fare
df_sample.base_passenger_fare.sum().compute() / 1e9

CPU times: user 30.2 ms, sys: 3.56 ms, total: 33.8 ms
Wall time: 509 ms


1.581799936

In [27]:
%%time
#tip
df_sample.tips.sum().compute() / 1e6

CPU times: user 29.7 ms, sys: 3.26 ms, total: 33 ms
Wall time: 383 ms


60.919496

In [28]:
%%time
df_sample.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

CPU times: user 42.6 ms, sys: 4.47 ms, total: 47.1 ms
Wall time: 864 ms


,sum,mean
hvfhs_license_num,,
HV0005,1.906722e+07,4.948551
HV0003,4.133530e+07,4.912592
HV0002,2.154587e+05,3.991971
HV0004,3.015095e+05,2.334387


## Partition size 1MB 

Runs are ~11X slower compared to 13MB partitions

In [29]:
df_sample = df_sample.repartition(partition_size="1MB").persist()

In [30]:
dask.utils.format_bytes(df_sample.partitions[0].memory_usage(deep=True).compute().sum())

'0.93 MiB'

In [31]:
%%time
#passanger fare
df_sample.base_passenger_fare.sum().compute() / 1e9

CPU times: user 126 ms, sys: 9.39 ms, total: 136 ms
Wall time: 5.87 s


1.581799808

In [33]:
%%time
#tip
df_sample.tips.sum().compute() / 1e6

CPU times: user 123 ms, sys: 8.34 ms, total: 131 ms
Wall time: 5.88 s


60.919496

In [34]:
%%time
df_sample.loc[lambda x: x.tips > 0].groupby("hvfhs_license_num").tips.agg(["sum", "mean"]).compute()

CPU times: user 138 ms, sys: 8.76 ms, total: 147 ms
Wall time: 6.8 s


,sum,mean
hvfhs_license_num,,
HV0005,1.906722e+07,4.948551
HV0003,4.133530e+07,4.912592
HV0002,2.154587e+05,3.991971
HV0004,3.015095e+05,2.334387


## Write 1MB partition data to parquet and csv

In [36]:
df_sample.to_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/");

In [41]:
df_sample.to_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/");

## Check read 

In [6]:
df_pq = dd.read_parquet("s3://coiled-datasets/uber-lyft-tlc-sample/parquet-ill/")

In [43]:
#beauty of parquet meta - get dtypes right
df_pq.dtypes

hvfhs_license_num       string[pyarrow]
dispatching_base_num    string[pyarrow]
originating_base_num    string[pyarrow]
request_datetime         datetime64[ns]
on_scene_datetime        datetime64[ns]
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
PULocationID                      int32
DOLocationID                      int32
trip_miles                      float32
trip_time                         int32
base_passenger_fare             float32
tolls                           float32
bcf                             float32
sales_tax                       float32
congestion_surcharge            float32
airport_fee                     float32
tips                            float32
driver_pay                      float32
shared_request_flag            category
shared_match_flag              category
access_a_ride_flag             category
wav_request_flag               category
wav_match_flag                 category
dtype: object

In [44]:
dask.utils.format_bytes(df_pq.partitions[0].memory_usage(deep=True).compute().sum())

'0.93 MiB'

### Try 100MB partitions

In [7]:
df_pq = df_pq.repartition(partition_size="100MB").persist()

In [8]:
%%time
#passanger fare
df_pq.base_passenger_fare.sum().compute() / 1e9

CPU times: user 19.1 ms, sys: 3.34 ms, total: 22.5 ms
Wall time: 137 ms


1.581799936

## Went from ~5.9 s to 137ms (~43X faster)

In [15]:
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*", 
                     dtype = {
                     "shared_request_flag": "category",
                     "shared_match_flag": "category",
                     "access_a_ride_flag": "category",
                     "wav_request_flag": "category",
                     "wav_match_flag": "category",    
                     }, 
                     parse_dates=["request_datetime", "on_scene_datetime", # this is not working well see error below
                                  "pickup_datetime", "dropoff_datetime"]
                    )

In [16]:
#notice the ints and floats are infered as 64
df_csv.dtypes

Unnamed: 0                        int64
hvfhs_license_num       string[pyarrow]
dispatching_base_num    string[pyarrow]
originating_base_num    string[pyarrow]
request_datetime         datetime64[ns]
on_scene_datetime        datetime64[ns]
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
PULocationID                      int64
DOLocationID                      int64
trip_miles                      float64
trip_time                         int64
base_passenger_fare             float64
tolls                           float64
bcf                             float64
sales_tax                       float64
congestion_surcharge            float64
airport_fee                     float64
tips                            float64
driver_pay                      float64
shared_request_flag            category
shared_match_flag              category
access_a_ride_flag             category
wav_request_flag               category
wav_match_flag                 category


In [17]:
#this is not working
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

The following columns failed to properly parse as dates:

- on_scene_datetime

This is usually due to an invalid value in that column. To
diagnose and fix it's recommended to drop these columns from the
`parse_dates` keyword, and manually convert them to dates later
using `dd.to_datetime`.

2023-06-30 17:23:06,843 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


```python
ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

The following columns failed to properly parse as dates:

- on_scene_datetime

This is usually due to an invalid value in that column. To
diagnose and fix it's recommended to drop these columns from the
`parse_dates` keyword, and manually convert them to dates later
using `dd.to_datetime`.


```

## If we just do 
```python
df_csv = dd.read_csv("s3://coiled-datasets/uber-lyft-tlc-sample/csv-ill/*",)
```

It will be infering date times and categories as strings, and the last category `wav_match_flag` because it has NaN is converting it to floats but the values are ["N", "Y"]

and this fails because of the dtype of `wav_match_flag`
```python
dask.utils.format_bytes(
    df_csv.memory_usage(deep=True).sum().compute()
)
```